In [2]:
import numpy as np
import pandas as pd

# import data

In [3]:
#read json
train = pd.read_json(r'C:\Users\User\Downloads\kaggle\whats_cooking\train.json')
test = pd.read_json(r'C:\Users\User\Downloads\kaggle\whats_cooking\test.json')
train.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


# preproccessing

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing.label import LabelEncoder

In [5]:
#transfer list to string
train['ingredients'] = train['ingredients'].apply(lambda x : ' '.join(x))
test['ingredients'] = test['ingredients'].apply(lambda x : ' '.join(x))

In [6]:
#transfer string to vector
token = TfidfVectorizer()
token.fit(train['ingredients'])
datax = token.transform(train['ingredients']).toarray()
testdatax = token.transform(test['ingredients']).toarray()

In [7]:
#transfer categorical data to numbers
lb = LabelEncoder()
train['cuisine'] = lb.fit_transform(train['cuisine'])

In [8]:
#number dict
category_dict = {i:j for i,j in enumerate(lb.classes_)}
category_dict

{0: 'brazilian',
 1: 'british',
 2: 'cajun_creole',
 3: 'chinese',
 4: 'filipino',
 5: 'french',
 6: 'greek',
 7: 'indian',
 8: 'irish',
 9: 'italian',
 10: 'jamaican',
 11: 'japanese',
 12: 'korean',
 13: 'mexican',
 14: 'moroccan',
 15: 'russian',
 16: 'southern_us',
 17: 'spanish',
 18: 'thai',
 19: 'vietnamese'}

In [9]:
#train/test split
datay = train['cuisine'].values
train_x,test_x,train_y,test_y = train_test_split(datax,datay,test_size=0.01,random_state=0)

# model(Keras)

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization

In [22]:
model_dl = Sequential()
model_dl.add(BatchNormalization(input_shape=(3034,)))
model_dl.add(Dense(2048,activation='relu'))
model_dl.add(Dropout(0.5))
model_dl.add(Dense(512,activation='relu'))
model_dl.add(Dropout(0.5))
model_dl.add(Dense(128,activation='relu'))
model_dl.add(Dropout(0.5))
model_dl.add(Dense(20,activation='softmax'))
model_dl.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 3034)              12136     
_________________________________________________________________
dense_14 (Dense)             (None, 2048)              6215680   
_________________________________________________________________
dropout_11 (Dropout)         (None, 2048)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 512)               1049088   
_________________________________________________________________
dropout_12 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               65664     
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)              

In [24]:
model_dl.compile('adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])

In [25]:
model_dl.fit(train_x,train_y,batch_size=1000,epochs=15,validation_data=[test_x,test_y])

Train on 39376 samples, validate on 398 samples
Epoch 1/15
39376/39376 [==============================] - 15s 387us/sample - loss: 1.8447 - sparse_categorical_accuracy: 0.4811 - val_loss: 2.6213 - val_sparse_categorical_accuracy: 0.7111
Epoch 2/15
39376/39376 [==============================] - 16s 402us/sample - loss: 1.0516 - sparse_categorical_accuracy: 0.7019 - val_loss: 2.5212 - val_sparse_categorical_accuracy: 0.6508
Epoch 3/15
39376/39376 [==============================] - 12s 292us/sample - loss: 0.8421 - sparse_categorical_accuracy: 0.7600 - val_loss: 2.4743 - val_sparse_categorical_accuracy: 0.7060
Epoch 4/15
39376/39376 [==============================] - 13s 320us/sample - loss: 0.7063 - sparse_categorical_accuracy: 0.7976 - val_loss: 2.5084 - val_sparse_categorical_accuracy: 0.6910
Epoch 5/15
39376/39376 [==============================] - 12s 305us/sample - loss: 0.6135 - sparse_categorical_accuracy: 0.8201 - val_loss: 2.5713 - val_sparse_categorical_accuracy: 0.6030
Epoch 6

In [26]:
pred = model_dl.predict_classes(testdatax)

In [27]:
pred_df = pd.DataFrame([])
pred_df['id'] = test['id']
pred_df['cuisine'] = pred
pred_df['cuisine'] = pred_df['cuisine'].map(category_dict)

sub = pd.read_csv(r'C:\Users\User\Downloads\kaggle\whats_cooking\sample_submission.csv')
sub = sub[['id']]
sub = sub.merge(pred_df)
sub.to_csv(r'C:\Users\User\Downloads\kaggle\whats_cooking\sample_submission.csv',index=False)

# dirty fitting

In [29]:
model_dl2 = Sequential()
model_dl2.add(BatchNormalization(input_shape=(3034,)))
model_dl2.add(Dense(2048,activation='relu'))
model_dl2.add(Dropout(0.5))
model_dl2.add(Dense(512,activation='relu'))
model_dl2.add(Dropout(0.5))
model_dl2.add(Dense(128,activation='relu'))
model_dl2.add(Dropout(0.5))
model_dl2.add(Dense(20,activation='softmax'))
model_dl2.compile('adam',loss='sparse_categorical_crossentropy',metrics=['sparse_categorical_accuracy'])
model_dl2.fit(datax,datay,batch_size=1000,epochs=15)

Epoch 1/15
39774/39774 [==============================] - 14s 355us/sample - loss: 1.8079 - sparse_categorical_accuracy: 0.4971
Epoch 2/15
39774/39774 [==============================] - 11s 286us/sample - loss: 1.0598 - sparse_categorical_accuracy: 0.6996
Epoch 3/15
39774/39774 [==============================] - 11s 271us/sample - loss: 0.8317 - sparse_categorical_accuracy: 0.7616
Epoch 4/15
39774/39774 [==============================] - 11s 268us/sample - loss: 0.7025 - sparse_categorical_accuracy: 0.7953
Epoch 5/15
39774/39774 [==============================] - 11s 275us/sample - loss: 0.5975 - sparse_categorical_accuracy: 0.8249
Epoch 6/15
39774/39774 [==============================] - 11s 284us/sample - loss: 0.5234 - sparse_categorical_accuracy: 0.8471
Epoch 7/15
39774/39774 [==============================] - 11s 274us/sample - loss: 0.4467 - sparse_categorical_accuracy: 0.8672
Epoch 8/15
39774/39774 [==============================] - 11s 268us/sample - loss: 0.3860 - sparse_categ

In [30]:
pred = model_dl2.predict_classes(testdatax)

In [31]:
pred_df = pd.DataFrame([])
pred_df['id'] = test['id']
pred_df['cuisine'] = pred
pred_df['cuisine'] = pred_df['cuisine'].map(category_dict)

sub = pd.read_csv(r'C:\Users\User\Downloads\kaggle\whats_cooking\sample_submission.csv')
sub = sub[['id']]
sub = sub.merge(pred_df)
sub.to_csv(r'C:\Users\User\Downloads\kaggle\whats_cooking\sample_submission.csv',index=False)

# submission

In [13]:
pred = model_dl.predict_classes(testdatax)

In [14]:
pred_df = pd.DataFrame([])
pred_df['id'] = test['id']
pred_df['cuisine'] = pred
pred_df['cuisine'] = pred_df['cuisine'].map(category_dict)

In [15]:
pred_df.head()

,id,cuisine
0,18009,southern_us
1,28583,southern_us
2,41580,italian
3,29752,cajun_creole
4,35687,italian


In [16]:
sub = pd.read_csv(r'C:\Users\User\Downloads\kaggle\whats_cooking\sample_submission.csv')
sub = sub[['id']]
sub = sub.merge(pred_df)
sub.to_csv(r'C:\Users\User\Downloads\kaggle\whats_cooking\sample_submission.csv',index=False)

### result :  0.79847 (236/1387) (17%)